# Lab 3.1 - Building AI Agents with watsonx.ai

This notebook demonstrates how to build a basic AI agent using watsonx.ai with custom tools.

## What You'll Learn

- Creating a simple agent architecture with watsonx.ai
- Implementing custom tools (RAG service and calculator)
- Building a planner-executor agent pattern
- Using Granite models for tool selection and answer generation
- Handling tool execution and error cases

## Agent Architecture

```
User Question
     |
     v
Planner (watsonx.ai)
  - Analyzes question
  - Selects tool
  - Determines arguments
     |
     v
Tool Executor
  - RAG Service
  - Calculator
     |
     v
Answer Generator (watsonx.ai)
  - Formats tool output
  - Generates user-friendly response
     |
     v
Final Answer
```

---

## 1. Setup and Installation

### Google Colab Compatibility

This notebook is designed to work seamlessly in both Google Colab and local environments.

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local environment")

In [ ]:
# Install dependencies
!pip install -q "ibm-watsonx-ai>=1.1.22" requests pydantic

## 2. Configure watsonx.ai Credentials

### How to Get Your Credentials

1. **API Key**:
   - Go to [IBM Cloud](https://cloud.ibm.com/iam/apikeys)
   - Click "Create an IBM Cloud API key"
   - Copy the API key

2. **Project ID**:
   - Go to [watsonx.ai](https://dataplatform.cloud.ibm.com/wx)
   - Open your project
   - Click "Manage" → "General" → Copy the Project ID

3. **URL**: Use regional endpoint (default: us-south)

In [ ]:
import os
from getpass import getpass

# Configuration for watsonx.ai
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

if not os.getenv("WATSONX_APIKEY"):
    WATSONX_APIKEY = getpass("Enter your watsonx.ai API Key: ")
else:
    WATSONX_APIKEY = os.getenv("WATSONX_APIKEY")

if not os.getenv("WATSONX_PROJECT_ID"):
    WATSONX_PROJECT_ID = getpass("Enter your watsonx.ai Project ID: ")
else:
    WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

# Model configuration
LLM_MODEL_ID = os.getenv("LLM_MODEL_ID", "ibm/granite-3-8b-instruct")

# Accelerator API URL (update this to your RAG service endpoint)
ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")

print("✓ Configuration loaded")
print(f"  Model: {LLM_MODEL_ID}")
print(f"  URL: {WATSONX_URL}")
print(f"  RAG API: {ACCELERATOR_API_URL}")

## 3. Initialize watsonx.ai Model

We'll create a watsonx.ai model instance for both planning and answer generation.

In [ ]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

# Set up credentials
creds = Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY)

# Configure model parameters
params = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 800,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.3,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

# Initialize the model
planner_model = Model(
    model_id=LLM_MODEL_ID,
    credentials=creds,
    project_id=WATSONX_PROJECT_ID,
    params=params,
)

print("✓ watsonx.ai model initialized successfully")

# Test the model
test_response = planner_model.generate_text("What is artificial intelligence?")
print(f"\nTest response: {test_response['results'][0]['generated_text'][:100]}...")

## 4. Define Custom Tools

We'll create two tools:

1. **RAG Service Tool** - Queries the accelerator RAG API
2. **Calculator Tool** - Safely evaluates arithmetic expressions

### RAG Service Tool

In [ ]:
import requests
import json
import time
from typing import Dict, Any

def rag_service_tool(question: str) -> Dict[str, Any]:
    """
    Call the accelerator RAG service with a question.
    
    This tool retrieves information from a knowledge base using
    Retrieval-Augmented Generation (RAG).
    
    Args:
        question: The question to ask the RAG service
    
    Returns:
        Dictionary containing:
        - answer: The RAG service answer
        - citations: Source citations (if available)
        - latency_ms: Response time in milliseconds
    """
    payload = {"question": question}
    start = time.time()
    
    try:
        resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
        latency_ms = int((time.time() - start) * 1000)
        resp.raise_for_status()
        
        data = resp.json()
        data.setdefault("latency_ms", latency_ms)
        return data
    except Exception as e:
        return {
            "error": str(e),
            "answer": f"Error calling RAG service: {str(e)}",
            "citations": [],
            "latency_ms": int((time.time() - start) * 1000)
        }


print("✓ RAG service tool defined")

### Calculator Tool

In [ ]:
import ast
import operator as op

# Safe calculator implementation using AST
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.Mod: op.mod,
}


def _eval_ast(node):
    """
    Safely evaluate an AST node.
    
    This prevents code injection by only allowing
    basic arithmetic operations.
    """
    if isinstance(node, ast.Num):  # Python 3.7 compatibility
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")


def calculator_tool(expression: str) -> str:
    """
    Safely evaluate a mathematical expression.
    
    Supports: +, -, *, /, %, ** (power)
    Does NOT support: variables, functions, imports
    
    Args:
        expression: Mathematical expression as a string
    
    Returns:
        Result as a string, or error message
    
    Examples:
        >>> calculator_tool("2 + 2")
        "4"
        >>> calculator_tool("10 * (5 + 3)")
        "80"
    """
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


print("✓ Calculator tool defined")

## 5. Build the Planner

The planner uses watsonx.ai to decide which tool to use based on the user's question.

In [ ]:
from pydantic import BaseModel

class ToolPlan(BaseModel):
    """
    Schema for the planner's output.
    
    Attributes:
        tool: Name of the tool to use ("rag_service" or "calculator")
        arguments: Dictionary of arguments for the tool
    """
    tool: str
    arguments: Dict[str, Any]


PLANNER_SYSTEM_PROMPT = """
You are a planner agent that decides which tool to use for a given question.

Available tools:
1. rag_service: Use this for knowledge-based questions about concepts, definitions, 
   explanations, or any question that requires accessing a knowledge base.
   Arguments: {"question": "the user's question"}

2. calculator: Use this for mathematical calculations and arithmetic operations.
   Arguments: {"expression": "mathematical expression like '2 + 2' or '10 * (5 + 3)'"}

Instructions:
- Analyze the user's question carefully
- Choose EXACTLY ONE tool
- Return ONLY valid JSON with no additional text
- Format: {"tool": "tool_name", "arguments": {"arg": "value"}}

Decision rules:
- If the question asks for calculation or contains math operators → use calculator
- If the question asks "what is", "explain", "describe", etc. → use rag_service
- When in doubt, prefer rag_service
"""


def plan_tool_call(user_input: str) -> ToolPlan:
    """
    Use watsonx.ai to plan which tool to call.
    
    Args:
        user_input: The user's question
    
    Returns:
        ToolPlan object containing tool name and arguments
    """
    user_prompt = f"""
User question: {user_input}

Respond with JSON only:
{{"tool": "tool_name", "arguments": {{"arg": "value"}}}}
"""
    
    prompt = f"{PLANNER_SYSTEM_PROMPT}\n\n{user_prompt}"
    
    # Generate response
    raw = planner_model.generate_text(prompt=prompt)
    text = raw["results"][0]["generated_text"].strip()
    
    # Parse JSON response
    try:
        plan_dict = json.loads(text)
    except json.JSONDecodeError:
        # Try to extract JSON from the response
        start = text.find("{")
        end = text.rfind("}") + 1
        if start >= 0 and end > start:
            plan_dict = json.loads(text[start:end])
        else:
            # Fallback: default to RAG service
            print(f"Warning: Could not parse planner output, defaulting to rag_service")
            plan_dict = {"tool": "rag_service", "arguments": {"question": user_input}}
    
    return ToolPlan(**plan_dict)


print("✓ Planner defined")

## 6. Build the Answer Generator

After tool execution, we use watsonx.ai to generate a user-friendly final answer.

In [ ]:
FINAL_ANSWER_SYSTEM = """
You are a helpful AI assistant powered by watsonx.ai.

Your task is to provide a clear, concise final answer to the user based on:
1. The original question
2. The tool that was used
3. The output from that tool

Guidelines:
- Be concise and direct
- If the tool output contains an error, explain it politely and suggest alternatives
- For RAG answers, incorporate citations if available
- For calculator results, explain the calculation briefly
- Maintain a helpful, professional tone
"""


def generate_final_answer(user_input: str, tool_name: str, tool_output: Any) -> str:
    """
    Generate a user-friendly final answer using watsonx.ai.
    
    Args:
        user_input: Original user question
        tool_name: Name of the tool that was used
        tool_output: Output from the tool
    
    Returns:
        Final answer as a string
    """
    # Format tool output for the prompt
    if isinstance(tool_output, dict):
        tool_output_str = json.dumps(tool_output, indent=2)
    else:
        tool_output_str = str(tool_output)
    
    context = f"""
User question: {user_input}

Tool used: {tool_name}

Tool output:
{tool_output_str}

Please provide a clear, helpful final answer to the user's question.
"""
    
    prompt = f"{FINAL_ANSWER_SYSTEM}\n\n{context}"
    
    # Generate response
    raw = planner_model.generate_text(prompt=prompt)
    return raw["results"][0]["generated_text"].strip()


print("✓ Answer generator defined")

## 7. Orchestrate the Agent

Now we'll tie everything together into a complete agent system.

In [ ]:
def run_agent_once(user_input: str, verbose: bool = True) -> Dict[str, Any]:
    """
    Run one complete agent cycle: plan → execute → answer.
    
    Args:
        user_input: The user's question
        verbose: Whether to print detailed execution info
    
    Returns:
        Dictionary containing:
        - question: Original question
        - tool: Tool that was used
        - tool_args: Arguments passed to the tool
        - tool_output: Raw tool output
        - final_answer: Generated final answer
        - execution_time_ms: Total execution time
    """
    start_time = time.time()
    
    if verbose:
        print("=" * 80)
        print(f"USER QUESTION: {user_input}")
        print("=" * 80)
    
    # Step 1: Plan which tool to use
    if verbose:
        print("\n[1/3] Planning...")
    
    plan = plan_tool_call(user_input)
    tool_name = plan.tool
    args = plan.arguments or {}
    
    if verbose:
        print(f"  → Selected tool: {tool_name}")
        print(f"  → Arguments: {json.dumps(args, indent=4)}")
    
    # Step 2: Execute the tool
    if verbose:
        print("\n[2/3] Executing tool...")
    
    if tool_name == "rag_service":
        question = args.get("question") or user_input
        tool_output = rag_service_tool(question)
    elif tool_name == "calculator":
        expr = args.get("expression") or user_input
        tool_output = calculator_tool(expr)
    else:
        tool_output = f"Unknown tool: {tool_name}"
    
    if verbose:
        if isinstance(tool_output, dict):
            print(f"  → Tool output: {json.dumps(tool_output, indent=4)[:200]}...")
        else:
            print(f"  → Tool output: {str(tool_output)[:200]}")
    
    # Step 3: Generate final answer
    if verbose:
        print("\n[3/3] Generating final answer...")
    
    final_answer = generate_final_answer(user_input, tool_name, tool_output)
    
    execution_time_ms = int((time.time() - start_time) * 1000)
    
    if verbose:
        print("\n" + "=" * 80)
        print("FINAL ANSWER:")
        print("=" * 80)
        print(final_answer)
        print("\n" + "=" * 80)
        print(f"Execution time: {execution_time_ms}ms")
        print("=" * 80)
    
    return {
        "question": user_input,
        "tool": tool_name,
        "tool_args": args,
        "tool_output": tool_output,
        "final_answer": final_answer,
        "execution_time_ms": execution_time_ms
    }


print("✓ Agent orchestration defined")

## 8. Test the Agent

Let's test our agent with different types of questions.

### Test 1: Knowledge Question (RAG)

In [ ]:
result_1 = run_agent_once("What is Retrieval-Augmented Generation and why do we use it?")

### Test 2: Mathematical Question (Calculator)

In [ ]:
result_2 = run_agent_once("What is 2 * (3 + 4)?")

### Test 3: Another Knowledge Question

In [ ]:
result_3 = run_agent_once("Explain the benefits of using watsonx.ai for enterprise AI")

## 9. Batch Testing

Test multiple questions and analyze the results.

In [ ]:
test_questions = [
    "What is RAG and why do we use it?",
    "Calculate 15 * 8 + 20",
    "What are the key features of IBM Granite models?",
    "Compute (100 - 25) / 5",
]

results = []

print("\n" + "=" * 80)
print("BATCH TESTING")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n[Test {i}/{len(test_questions)}]")
    result = run_agent_once(question, verbose=False)
    results.append(result)
    print(f"Q: {question}")
    print(f"Tool: {result['tool']}")
    print(f"A: {result['final_answer'][:150]}...")
    print(f"Time: {result['execution_time_ms']}ms")

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total questions: {len(results)}")
print(f"RAG calls: {sum(1 for r in results if r['tool'] == 'rag_service')}")
print(f"Calculator calls: {sum(1 for r in results if r['tool'] == 'calculator')}")
print(f"Avg execution time: {sum(r['execution_time_ms'] for r in results) // len(results)}ms")

## 10. Interactive Mode

Ask your own questions!

In [ ]:
def ask_agent(question: str):
    """
    Simple interface to ask the agent a question.
    
    Args:
        question: Your question
    
    Returns:
        The agent's answer
    """
    result = run_agent_once(question, verbose=True)
    return result['final_answer']


# Try it out!
# ask_agent("Your question here")

## 11. Summary and Key Takeaways

### What We Built

1. **Planner-Executor Agent Pattern**: A two-stage agent that first plans, then executes
2. **Custom Tools**: RAG service and calculator with proper error handling
3. **JSON-based Communication**: Structured communication between planner and executor
4. **Answer Generation**: LLM-powered final answer formatting

### Key Components

- **Planner**: Uses watsonx.ai to decide which tool to use
- **Tools**: Extensible tool system (RAG and calculator)
- **Executor**: Runs the selected tool with appropriate arguments
- **Generator**: Creates user-friendly final answers

### Advantages of This Approach

1. **Explicit Tool Selection**: Clear decision-making process
2. **Extensibility**: Easy to add new tools
3. **Error Handling**: Robust error management at each step
4. **Auditability**: Complete execution trace for debugging
5. **Flexibility**: Can adapt to different use cases

### Limitations

1. **Single Tool**: Can only use one tool per question
2. **No Memory**: Doesn't remember previous interactions
3. **No Iteration**: Can't retry or refine answers
4. **Fixed Tools**: Tool list is hardcoded

### Extending This Agent

To make this agent more powerful, you could:

1. **Add More Tools**:
   - Web search
   - Database query
   - File operations
   - API calls

2. **Add Memory**:
   - Conversation history
   - User preferences
   - Session state

3. **Multi-Step Planning**:
   - Use multiple tools in sequence
   - Implement chain-of-thought reasoning
   - Add reflection and self-correction

4. **Better Tool Selection**:
   - Fine-tune the planner model
   - Add few-shot examples
   - Implement confidence scoring

### Comparison with Other Approaches

| Feature | Basic Agent | CrewAI | LangGraph |
|---------|-------------|--------|----------|
| Complexity | Low | Medium | Medium-High |
| Tool Support | Manual | Built-in | Built-in |
| Multi-Agent | No | Yes | Yes |
| State Management | Manual | Automatic | Explicit |
| Learning Curve | Easy | Medium | Medium |
| Flexibility | High | Medium | Very High |

### Best Practices

1. **Clear Tool Descriptions**: Help the planner make better decisions
2. **Structured Prompts**: Use consistent prompt formats
3. **Error Handling**: Always handle tool failures gracefully
4. **Logging**: Track all decisions for debugging
5. **Testing**: Test with diverse question types

### Next Steps

- Try the **CrewAI notebook** for multi-agent collaboration
- Explore **LangGraph** for complex stateful workflows
- Experiment with **Langflow** for visual agent building
- Build your own custom tools and agents

---

**Course**: Multi-Agent Systems with watsonx.ai  
**Lab**: 3.1 - Basic Agent with watsonx.ai  
**Platform**: Compatible with Google Colab and local environments